In [1]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2
import seaborn as sns

## Load the training data into feature matrix, class labels, and event ids, and separate the dataset:

In [2]:
from implementations import *

In [3]:
from proj1_helpers import *
DATA_TRAIN_PATH = '../data/train.csv' # TODO: download train data and supply path here 
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)


In [4]:
DATA_TEST_PATH = '../data/test.csv' # TODO: download train data and supply path here 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [5]:
logistic = False
tX_list, ids_list, y_list = separate_dataset(tX, ids, y, logistic)
tX_test_list, ids_test_list = separate_dataset(tX_test, ids_test) 

In [6]:
tX_list, tX_test_list, y_list = clean_data(tX_list, tX_test_list, y_list)

## GRID SEARCH RIDGE REGRESSION

In [8]:
function = reg_logistic_regression

degrees = range(1, 5)
lambdas = np.logspace(-6, 0, 3)
gammas = np.logspace(-3, 0, 3)

degree_vec = []
lambda_vec = []
gamma_vec = []

for i in range(6):
    print('Dataset part {l}:'.format(l = i))
    rmse_te, BestDeg, BestLambda, BestGamma = grid_search(y_list[i], tX_list[i], function, 4, degrees, lambdas, gammas, dataset = i)
    degree_vec.append(BestDeg)
    lambda_vec.append(BestLambda)
    gamma_vec.append(BestGamma)
    print('     Best degree: {d}'.format(d = BestDeg))
    print('     Best lambda: {m}'.format(m = BestLambda))
    print('     Best gamma: {m}'.format(m = BestGamma))
    print('     Loss: {lo}'.format(lo = rmse_te))

print(degree_vec)
print(lambda_vec)
print(gamma_vec)

Dataset part 0:
(array([13, 13, 13, 13, 13]), array([0, 0, 0, 0, 0]), array([1, 3, 5, 6, 9]))
     Best degree: 14
     Best lambda: 2.9935772947204904e-10
     Loss: 6173.5
Dataset part 1:
(array([14]), array([0]), array([11]))
     Best degree: 15
     Best lambda: 1.7301957388458942e-05
     Loss: 14873.75
Dataset part 2:
(array([6]), array([0]), array([15]))
     Best degree: 7
     Best lambda: 0.001389495494373136
     Loss: 1723.75
Dataset part 3:
(array([11]), array([0]), array([15]))
     Best degree: 12
     Best lambda: 0.001389495494373136
     Loss: 13809.0
Dataset part 4:
(array([4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]), array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15]))
     Best degree: 5
     Best lambda: 1e-10
     Loss: 997.0
Dataset part 5:
(array([15]), array([0]), array([14]))
     Best degree: 16
     Best lambda: 0.00046415888336127724
     Loss: 13911.5


In [9]:
weights_list = []
loss_list = []
mat_tX_test_list = []

for i in range(6):
    mat_tX, mat_tX_test = build_poly_log(tX_list[i], degree_vec[i], tX_test_list[i], i)
    w, l = ridge_regression(y_list[i], mat_tX, lambda_vec[i])
    weights_list.append(w)
    loss_list.append(l)
    mat_tX_test_list.append(mat_tX_test)

In [10]:
# Eval

y_pred_list = separated_eval(weights_list, mat_tX_test_list, logistic) 

y_pred = np.concatenate((y_pred_list[0], y_pred_list[1], y_pred_list[2], y_pred_list[3], y_pred_list[4], y_pred_list[5]))
ids_test_sub = np.concatenate((ids_test_list[0], ids_test_list[1], ids_test_list[2], ids_test_list[3], ids_test_list[4], ids_test_list[5]))

OUTPUT_PATH = 'result.csv'
create_csv_submission(ids_test_sub, y_pred, OUTPUT_PATH)